# Домашнее задание № 3

### Выполнил Кузнецов С.И.

Реализуем следующий гибридный алгоритм:
1. Обучим модель 3мя методами:
    * SVD
    * BaselineOnly
    * KNNWithMeans
2. Обучим мета-алгоритм, который присвоит веса каждой оценке
3. Итоговая оценка - средневзвешенная оценка этих 3х алгоритмов

In [1]:
import numpy as np
import pandas as pd
from surprise import SVD, BaselineOnly, KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from lightfm.datasets import fetch_movielens
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

Загружаем данные

In [2]:
movielens = fetch_movielens()
train = movielens['train']
test = movielens['test']

In [3]:
def sparse_matrix_to_dataframe(sparse_matrix):
    '''
    Преобразует разреженную матрицу к dataframe
    '''
    return pd.DataFrame({
        'uid': sparse_matrix.nonzero()[0],
        'iid': sparse_matrix.nonzero()[1],
        'rating': sparse_matrix.data
    })

Разбиваем выборку на обучающую, тестовую для мета-алгоритма и итоговую тестовую

In [4]:
reader = Reader(rating_scale=(1.0, 5.0))
train_test_df = Dataset.load_from_df(sparse_matrix_to_dataframe(train), reader)
_, test_total_df = train_test_split(Dataset.load_from_df(sparse_matrix_to_dataframe(test), reader), test_size=1.0)
train_df, test_meta_df = train_test_split(train_test_df, test_size=.2)

**Обучаем алгоритмы**

Алгоритм SVD:

In [5]:
%%time
algo_SVD = SVD()
algo_SVD.fit(train_df)

CPU times: user 17.8 s, sys: 70.6 ms, total: 17.9 s
Wall time: 20.5 s


Алгоритм BaselineOnly

In [6]:
%%time
algo_BaselineOnly = BaselineOnly()
algo_BaselineOnly.fit(train_df)

Estimating biases using als...
CPU times: user 559 ms, sys: 3.76 ms, total: 563 ms
Wall time: 711 ms


Алгоритм KNNWithMeans

In [7]:
%%time
algo_KNNWithMeans = KNNWithMeans()
algo_KNNWithMeans.fit(train_df)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 1.56 s, sys: 51 ms, total: 1.61 s
Wall time: 1.98 s


Построим предсказания этих алгортимов на обучающей выборке для мета-алгоритма и тестовой выборке

In [8]:
def get_estimates_all_algo(df, algo_SVD, algo_BaselineOnly, algo_KNNWithMeans):
    '''
    Возвращает датафрейм с оценками по всем алгоритмам
    '''
    estimates = []
    scores_SVD = []
    scores_BaselineOnly = []
    scores_KNNWithMeans = []

    for uid, iid, estimate in df:
        scores_SVD.append(algo_SVD.predict(uid=uid, iid=iid).est)
        scores_BaselineOnly.append(algo_BaselineOnly.predict(uid=uid, iid=iid).est)
        scores_KNNWithMeans.append(algo_KNNWithMeans.predict(uid=uid, iid=iid).est)
        estimates.append(estimate)
    
    res = pd.DataFrame({
        'estimates': estimates,
        'score_SVD': scores_SVD,
        'score_BaselineOnly': scores_BaselineOnly,
        'score_KNNWithMeans': scores_KNNWithMeans
    })

    return res

In [9]:
meta_train_scores = get_estimates_all_algo(test_meta_df, algo_SVD, algo_BaselineOnly, algo_KNNWithMeans)
meta_train_scores.head()

,estimates,score_SVD,score_BaselineOnly,score_KNNWithMeans
0,4.0,4.462999,4.350391,4.272162
1,4.0,3.401721,3.320642,3.571164
2,4.0,3.292835,3.289137,3.807549
3,3.0,3.752698,3.756246,3.777072
4,4.0,2.639299,2.827124,2.338617


In [10]:
test_scores = get_estimates_all_algo(test_total_df, algo_SVD, algo_BaselineOnly, algo_KNNWithMeans)
test_scores.head()

,estimates,score_SVD,score_BaselineOnly,score_KNNWithMeans
0,3.0,3.051589,3.670966,3.830858
1,5.0,4.361444,4.221002,4.653053
2,3.0,3.503591,3.882559,3.757264
3,4.0,3.298950,3.363793,3.471830
4,4.0,4.060819,3.922160,3.510769


Строим линейную регрессию на 3х алгоритмах.

In [11]:
X_train = meta_train_scores[['score_SVD', 'score_BaselineOnly', 'score_KNNWithMeans']]
y_train = meta_train_scores.estimates
X_test = test_scores[['score_SVD', 'score_BaselineOnly', 'score_KNNWithMeans']]
y_test = test_scores.estimates

In [12]:
meta_model = LinearRegression()
meta_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
meta_model.coef_

array([0.59215656, 0.18945408, 0.2443104 ])

Проверим точность гибридного алгоритма по метрике RMSE

In [14]:
total_model_RMSE = math.sqrt(mean_squared_error(y_test, meta_model.predict(X_test)))
SVD_RMSE = accuracy.rmse(algo_SVD.test(test_total_df), verbose = False)
BaselineOnly_RMSE = accuracy.rmse(algo_BaselineOnly.test(test_total_df), verbose = False)
KNNWithMeans_RMSE = accuracy.rmse(algo_KNNWithMeans.test(test_total_df), verbose = False)

print('RMSE алгоритма SVD: {:.4f}'.format(SVD_RMSE))
print('RMSE алгоритма BaselineOnly: {:.4f}'.format(BaselineOnly_RMSE))
print('RMSE алгоритма KNNWithMeans: {:.4f}'.format(KNNWithMeans_RMSE))
print('RMSE гибридного алгоритма: {:.4f}'.format(total_model_RMSE))

RMSE алгоритма SVD: 0.9645
RMSE алгоритма BaselineOnly: 0.9677
RMSE алгоритма KNNWithMeans: 0.9775
RMSE гибридного алгоритма: 0.9574


**Вывод:** Применение гибридных алгоритмов повышает качество рекомендаций.